# 3 | Deep Learning with PyTorch



## Tensors

### The Creation of Tensors


In [1]:
import torch as th
import numpy as np

a = th.FloatTensor(3, 2)
a

/Users/kckp856/Workspace/Deep-Reinforcement-Learning/.venv/lib/python3.14/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


ModuleNotFoundError: No module named 'numpy'